# Lecture 7: Neural Architecture Search (Part I)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gaurav-redhat/transformer_problems/blob/efficientml-course/efficientml_course/07_neural_architecture_search_1/demo.ipynb)

Implementing a simple differentiable NAS (DARTS-style).


In [ ]:
!pip install torch -q
import torch
import torch.nn as nn
import torch.nn.functional as F

# DARTS-style differentiable NAS
# Instead of discrete choice, use soft weighting

class MixedOp(nn.Module):
    """Mix of candidate operations with learnable weights"""
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.ops = nn.ModuleList([
            nn.Conv2d(in_ch, out_ch, 3, padding=1),  # Conv 3x3
            nn.Conv2d(in_ch, out_ch, 5, padding=2),  # Conv 5x5
            nn.MaxPool2d(3, stride=1, padding=1),     # MaxPool
            nn.Identity(),                            # Skip
        ])
        # Architecture weights (learnable)
        self.alpha = nn.Parameter(torch.zeros(4))
    
    def forward(self, x):
        weights = F.softmax(self.alpha, dim=0)
        out = sum(w * op(x) for w, op in zip(weights, self.ops))
        return out

# Demo
mixed = MixedOp(32, 32)
x = torch.randn(1, 32, 8, 8)
y = mixed(x)

print("Architecture weights (before training):")
print(f"  Conv3x3: {F.softmax(mixed.alpha, dim=0)[0]:.3f}")
print(f"  Conv5x5: {F.softmax(mixed.alpha, dim=0)[1]:.3f}")
print(f"  MaxPool: {F.softmax(mixed.alpha, dim=0)[2]:.3f}")
print(f"  Skip:    {F.softmax(mixed.alpha, dim=0)[3]:.3f}")
print("\n🎯 During training, weights learn which operation is best!")
